In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Smoothie_King/adjust_for_pending_stores'

In [2]:
writer_folder="./output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [3]:
excel_output=pd.ExcelFile("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20191028/SmoothieKing_TA_of_3_miles_zips_JL_2019-10-29.xlsx")
excel_output.sheet_names


['summary_by_TA', 'output_TA_by_store', 'zip_TA', 'summary_by_store_count']

In [4]:
df_by_store=excel_output.parse("output_TA_by_store",dtype=str)
df_by_store.shape

(947, 18)

In [5]:
new_store_excel=pd.ExcelFile("/home/jian/Projects/Smoothie_King/SK_celery/force_store_list/SK_forced_store_list_20191029.xlsx")
print(new_store_excel.sheet_names)
df_new_stores=new_store_excel.parse("opening",dtype=str)
new_store_list=df_new_stores['storenumber'].unique().tolist()
print(len(new_store_list))
print(new_store_list)

['opening', 'closed']
8
['1681', '1769', '1793', '1539', '1707', '1802', '1747', '1789']


In [6]:
df_by_store[df_by_store['storenumber'].isin(new_store_list)]

# All of the new stores need to be added

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,used_lat,used_long,zips_3_miles,TA
905,1681,"7949 Dani Drive, Unit #120",Fort Myers,FL,33966,2019-10-04 00:00:00,26.6034622,-81.8101998,Franchised,Nasser Naura,NaN,Y,New,NaN,26.6034622,-81.8101998,['33966'],"3_['0821', '0824', '1681']"
932,1769,4925 Walzem Road,San Antonio,TX,78218,2019-10-07 00:00:00,29.510673,-98.393281,Franchised,Matthew Coriell,NaN,Y,New,NaN,29.510673,-98.393281,"['78218', '78239', '78265', '78217', '78284']","3_['1509', '1643', '1769']"
940,1793,5009 Miller Road,Flint Township,MI,48507,2019-10-08 00:00:00,42.975289,-83.772766,Franchised,Christopher Klebba,NaN,Y,New,NaN,42.975289,-83.772766,"['48554', '48473', '48532', '48507', '48553']",1_['1793']


In [7]:
df_new_stores=df_new_stores[~df_new_stores['storenumber'].isin(df_by_store['storenumber'].tolist())]
df_new_stores

,storenumber
3,1539
4,1707
5,1802
6,1747
7,1789


In [8]:
TA_name=df_by_store[['storenumber','TA']].drop_duplicates()
TA_name_count=TA_name.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
TA_name_list=TA_name.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
TA_name=pd.merge(TA_name_count,TA_name_list,on="TA",how="outer")
TA_name['store_count']=TA_name['store_count'].apply(lambda x: str(x))
TA_name['store_list']=TA_name['store_list'].apply(lambda x: str(x))
TA_name['TA_new_name']=TA_name['store_count']+"_"+TA_name['store_list']
TA_name_dict=TA_name.set_index("TA").to_dict()['TA_new_name']


df_by_store['TA']=df_by_store['TA'].apply(lambda x: TA_name_dict[x])
df_by_store=df_by_store.rename(columns={"TA":"TA_new"})


In [9]:
zip_TA=excel_output.parse("zip_TA",dtype=str)
zip_TA['TA']=zip_TA['TA'].apply(lambda x: TA_name_dict[x])
zip_TA=zip_TA.rename(columns={"TA":"TA_new"})

# The automatic run region is not updated
del zip_TA['Co-Op/Regional']

In [10]:
region_by_store_file=pd.ExcelFile("/home/jian/Projects/Smoothie_King/adjust_for_pending_stores/region_mapping/Updated RAF List - Number of Opened Stores-2019-09-13.xlsx")
mapping_store_region=region_by_store_file.parse('All Stores',dtype=str)
mapping_store_region=mapping_store_region[['Store: Store Number  ↑','Co-Op/Regional  ↑']]
mapping_store_region.columns=['store','Co-Op/Regional']
mapping_store_region['Co-Op/Regional']=mapping_store_region['Co-Op/Regional'].fillna("nan")
mapping_store_region['store']=mapping_store_region['store'].apply(lambda x: x.replace("SK-",""))
print(mapping_store_region['store'].apply(lambda x: len(x)).unique())

zip_TA=pd.merge(zip_TA,mapping_store_region,on="store",how="left")
zip_TA['Co-Op/Regional']=zip_TA['Co-Op/Regional'].fillna("Unknown")

[4]


In [11]:
df_new_stores

,storenumber
3,1539
4,1707
5,1802
6,1747
7,1789


In [12]:
dict_new_stores={}

# 1539	1501A Cornerside Boulevard Vienna, VA 22182
dict_new_stores.update({"1539":{"GoogleAddress":"1501a Cornerside Blvd, Vienna, VA 22182","Google_LatLng":[38.9295492,-77.2445564]}})

# 1707	105 Kennings Road, 105-2A Crosby, TX, 77532
dict_new_stores.update({"1707":{"GoogleAddress":"105 Kennings Rd #2A, Crosby, TX 77532","Google_LatLng":[29.8899358,-95.0627069]}})

# 1802	18602 Kuykendahl Rd, Suite #400 Spring, TX 77379
dict_new_stores.update({"1802":{"GoogleAddress":"18602 Kuykendahl Rd #400, Spring, TX 77379","Google_LatLng":[30.0408301,-95.4886038]}})

# 1747	1010-H Beards Hill Rd Aberdeen, MD, 21001
dict_new_stores.update({"1747":{"GoogleAddress":"1010 Beards Hill Rd, Aberdeen, MD 21001","Google_LatLng":[39.5212782,-76.181853]}})

# 1789	1309 W Fairmont Parkway Suite O La Porte, TX 77571
dict_new_stores.update({"1789":{"GoogleAddress":"1309 W Fairmont Pkwy Suite O, La Porte, TX 77571","Google_LatLng":[29.6512853,-95.0316923]}})



In [13]:
df_new_stores['GoogleAddress']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['GoogleAddress'])
df_new_stores['GoogleLatitude']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][0])
df_new_stores['GoogleLongitude']=df_new_stores['storenumber'].apply(lambda x: dict_new_stores[x]['Google_LatLng'][1])


In [14]:
df_new_stores['zips_3_miles']=np.nan

for ind,row in df_new_stores.iterrows():
    store_center=(row['GoogleLatitude'],row['GoogleLongitude'])
    zip_list=[]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],unit="mi")
        if dist<=3:
            zip_list.extend([zip_cd])
    df_new_stores.loc[ind,'zips_3_miles']=str(zip_list)


In [15]:
df_new_stores

,storenumber,GoogleAddress,GoogleLatitude,GoogleLongitude,zips_3_miles
3,1539,"1501a Cornerside Blvd, Vienna, VA 22182",38.929549,-77.244556,"['22067', '22107', '22027', '22102', '22103', ..."
4,1707,"105 Kennings Rd #2A, Crosby, TX 77532",29.889936,-95.062707,['77532']
5,1802,"18602 Kuykendahl Rd #400, Spring, TX 77379",30.040830,-95.488604,"['77068', '77379', '77388', '77273', '77290']"
6,1747,"1010 Beards Hill Rd, Aberdeen, MD 21001",39.521278,-76.181853,"['21001', '21005']"
7,1789,"1309 W Fairmont Pkwy Suite O, La Porte, TX 77571",29.651285,-95.031692,"['77572', '77571', '77507']"


In [16]:
cols=zip_TA.columns.tolist()
zip_ta_new_stores=pd.DataFrame(columns=zip_TA.columns.tolist())
for ind, row in df_new_stores.iterrows():
    store=row['storenumber']
    zip_list=eval(row['zips_3_miles'])
    df=pd.DataFrame({"zip":zip_list},index=[store]*len(zip_list)).reset_index().rename(columns={"index":"store"})
    df['TA_new']="new_stores"
    df['Primary_DMA']="new_stores"
    df['Co-Op/Regional']="Unknown"
    zip_ta_new_stores=zip_ta_new_stores.append(df)
zip_ta_new_stores=zip_ta_new_stores[cols]

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [17]:
zip_TA=zip_TA.append(zip_ta_new_stores)
df_TA_zips=zip_TA.copy()

In [18]:
df_TA_zips[df_TA_zips['Co-Op/Regional']=="nan"]

,store,TA_new,zip,Primary_DMA,Co-Op/Regional
124,0016,1_['0016'],70609,LAKE CHARLES,nan
125,0016,1_['0016'],70606,LAKE CHARLES,nan
126,0016,1_['0016'],70605,LAKE CHARLES,nan
127,0016,1_['0016'],70607,LAKE CHARLES,nan
195,0032,"3_['0032', '1146', '1686']",70364,NEW ORLEANS,nan
...,...,...,...,...,...
6459,1812,1_['1812'],60544,CHICAGO,nan
6460,1841,"4_['0271', '0995', '1532', '1841']",75083,DALLAS-FT. WORTH,nan
6461,1841,"4_['0271', '0995', '1532', '1841']",75080,DALLAS-FT. WORTH,nan
6462,1841,"4_['0271', '0995', '1532', '1841']",75252,DALLAS-FT. WORTH,nan


In [19]:
for region,df_group in df_TA_zips.groupby("Co-Op/Regional"):
    if region!="UnknowRegion":
        df_group=df_group[['zip','Co-Op/Regional']].drop_duplicates()
        df_group.to_csv(writer_folder+"zips_for_the_%s.csv"%region,index=False)
    else:
        # df_austin=df_group[df_group['store'].isin(Austin_stores['store'].tolist())]
        df_unknown=df_group[~df_group['store'].isin(Austin_stores['store'].tolist())]

        # df_austin.to_csv(writer_folder+"zips_for_the_%s.csv"%"AustinDMA",index=False)
        df_unknown.to_csv(writer_folder+"zips_for_the_%s.csv"%"UnknownRegion",index=False)
df_output_overall_zips=df_TA_zips[['zip']].drop_duplicates()
df_output_overall_zips.to_csv(writer_folder+"overall_zips_in_TA.csv",index=False)

In [20]:
writer=pd.ExcelWriter(writer_folder+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_TA_zips.to_excel(writer,"df_TA_zips",index=False)
df_new_stores.to_excel(writer,"new_stores_add_on",index=False)
df_by_store.to_excel(writer,"df_by_store",index=False)
rfa_mapping=pd.read_excel("/home/jian/Projects/Smoothie_King/SK_celery/RFA_region_20190128.xlsx",dtype=str)
rfa_mapping.to_excel(writer,"RFA_region_20190128",index=False)
writer.save()

In [21]:
writer_folder

'./output_2019-10-29/'

In [22]:
sorted(df_TA_zips['Co-Op/Regional'].unique().tolist())

['Atlanta',
 'Austin',
 'Baton Rouge',
 'Biloxi',
 'Charlotte',
 'Cincinnati',
 'Dallas-Fort Worth',
 'Houston',
 'Jackson',
 'Kansas City',
 'Memphis',
 'Miami-Ft. Lauderdale',
 'Mobile-Pensacola-Destin',
 'Nashville',
 'New Orleans',
 'Orlando',
 'San Antonio',
 'St. Louis',
 'Tampa',
 'Unknown',
 'Washington DC',
 'nan']